In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [3]:
# 加载和准备数据
data = pd.read_csv('earthquake.csv')
data['Date'] = pd.to_datetime(data['Date'])
data['Time'] = pd.to_datetime(data['Time'], format='%H:%M:%S').dt.time
data['Timestamp'] = data.apply(lambda row: pd.Timestamp(f"{row['Date']} {row['Time']}"), axis=1)
data['Timestamp'] = data['Timestamp'].view('int64') // 10**9 #Unix时间戳
features = data[['Timestamp', 'Longitude', 'Latitude']]
targets = data[['Depth', 'Magnitude']]

In [4]:
X_np=features.values
y_np=targets.values

In [5]:
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
scaler_x.fit(X_np)
scaler_y.fit(y_np)

# 使用相同的参数对训练数据和测试数据进行归一化
X_normalized = scaler_x.transform(X_np)
y_normalized = scaler_y.transform(y_np)

In [6]:
def create_sliding_windows(X, Y, window_size):
    X_windows, Y_windows = [], []
    for i in range(len(X) - window_size):
        # 从X中提取窗口
        X_window = X[i:i+window_size]
        # 从Y中提取紧接着窗口的下一个值作为标签
        Y_window = Y[i+window_size]
        X_windows.append(X_window)
        Y_windows.append(Y_window)
    return np.array(X_windows), np.array(Y_windows)

window_size = 20
X_windows, Y_windows = create_sliding_windows(X_normalized, y_normalized, window_size)

In [7]:
X = torch.tensor(X_windows, dtype=torch.float32)
Y = torch.tensor(Y_windows, dtype=torch.float32)

In [8]:
# 确定测试集的大小
test_size = 0.2

# 计算测试集应有的数据点数量
num_data_points = len(X)
num_test_points = int(num_data_points * test_size)

# 计算测试集和训练集的分割点
split_point = num_data_points - num_test_points

# 按顺序划分数据为训练集和测试集
X_train = X[:split_point]
X_test = X[split_point:]
y_train = Y[:split_point]
y_test = Y[split_point:]

In [9]:
X_train.shape

torch.Size([18714, 20, 3])

In [10]:
# 创建DataLoader
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

In [11]:
batch_size=64
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

In [25]:
# 定义位置编码
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

# 定义Transformer模型
class TransformerModel(nn.Module):
    def __init__(self, input_dim, seq_len, hidden_dim, output_dim, num_layers, dropout_prob):
        super(TransformerModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.pos_encoder = PositionalEncoding(hidden_dim, dropout_prob)

        # 由于3是质数，我们可以选择将输入特征投影到更高的维度
        self.embedding = nn.Linear(input_dim, hidden_dim)

        encoder_layers = TransformerEncoderLayer(d_model=hidden_dim, nhead=8, 
                                                 dim_feedforward=hidden_dim, dropout=dropout_prob)
        self.transformer_encoder = TransformerEncoder(encoder_layer=encoder_layers, num_layers=num_layers)

        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        x = self.embedding(x)  # 将输入特征投影到更高的维度
        x = self.dropout(x)
        x = self.pos_encoder(x)
        x = x.permute(1, 0, 2)  # Transformer expects (Seq Len, Batch, Features)
        out = self.transformer_encoder(x)
        out = self.fc_out(out[-1, :, :])  # 取最后一个时间步
        return out

# 初始化模型参数
input_dim = 3  # 时间、经度、纬度
seq_len = 20   # 序列长度
hidden_dim = 64  # 隐藏层的维度
num_layers = 2  # Transformer层的数量
output_dim = 2  # 震深和震级
dropout_prob = 0.5  # Dropout概率

# 创建模型
model = TransformerModel(input_dim, seq_len, hidden_dim, output_dim, num_layers, dropout_prob)

# 检查CUDA是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 将模型移到GPU
model = model.to(device)

# 定义损失函数和优化器
criterion = nn.MSELoss()  # 因为是回归问题
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 设置训练轮数
num_epochs = 200

In [16]:
best_loss = np.inf
best_model_path = 'best_transformer_model.pth'

for epoch in range(num_epochs):
    model.train()  # 确保模型处于训练模式
    epoch_loss = 0.0  # 用于累积整个epoch的损失

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)  # 移动数据到GPU
        
        optimizer.zero_grad()  # 清除过往梯度
        
        outputs = model(inputs)  # 前向传播
        
        loss = criterion(outputs, targets)  # 计算损失
        loss.backward()  # 后向传播，计算梯度
        optimizer.step()  # 更新权重

        epoch_loss += loss.item() * inputs.size(0)  # 累积损失

    # 计算整个epoch的平均损失
    epoch_loss /= len(train_loader.dataset)
    
    # 打印训练进度
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    
    # 保存最佳模型
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(model.state_dict(), best_model_path)
torch.save(model.state_dict(), 'last_transformer_model.pth')

print("Training complete. Best loss: {:.4f}".format(best_loss))

Epoch [10/200], Loss: 0.0226
Epoch [20/200], Loss: 0.0225
Epoch [30/200], Loss: 0.0225
Epoch [40/200], Loss: 0.0224
Epoch [50/200], Loss: 0.0223
Epoch [60/200], Loss: 0.0223
Epoch [70/200], Loss: 0.0223
Epoch [80/200], Loss: 0.0223
Epoch [90/200], Loss: 0.0223
Epoch [100/200], Loss: 0.0222
Epoch [110/200], Loss: 0.0222
Epoch [120/200], Loss: 0.0222
Epoch [130/200], Loss: 0.0222
Epoch [140/200], Loss: 0.0222
Epoch [150/200], Loss: 0.0222
Epoch [160/200], Loss: 0.0222
Epoch [170/200], Loss: 0.0222
Epoch [180/200], Loss: 0.0222
Epoch [190/200], Loss: 0.0222
Epoch [200/200], Loss: 0.0222
Training complete. Best loss: 0.0222


In [31]:
def evaluate_model(model, test_loader):
    model.eval() # 将模型设置为评估模式
    predictions = []
    actuals = []
    with torch.no_grad(): # 不计算梯度
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device) # 移动数据到GPU
            outputs = model(inputs)
            predictions.append(outputs.cpu().numpy())
            actuals.append(targets.cpu().numpy())
    
    predictions = np.vstack(predictions)
    actuals = np.vstack(actuals)
    
    # 反标准化以比较实际值
    predictions = scaler_y.inverse_transform(predictions)
    actuals = scaler_y.inverse_transform(actuals)
    
    # 计算均方根误差
    mse = mean_squared_error(actuals, predictions)
    rmse = np.sqrt(mse)
    
    # 计算平均绝对误差
    mae = mean_absolute_error(actuals, predictions)

    return rmse, mae

In [33]:
# 加载最佳模型
model.load_state_dict(torch.load(best_model_path))
model.to(device)  # 确保模型在正确的设备上

# 评估模型
rmse, mae = evaluate_model(model, test_loader)

print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Error (MAE): {mae}')

Root Mean Squared Error (RMSE): 9.594
Mean Absolute Error (MAE): 8.762
